In [8]:
import requests
import pandas as pd

In [9]:
base_item_url = "https://poe.ninja/api/data/itemoverview?league={league}&type={item}"
base_currency_url = "https://poe.ninja/api/data/currencyoverview?league={league}&type={item}"
league = "Ancestor"

In [10]:
def template_replace(template_input, item_type):
    '''
    Replace template url to be an f-string
    '''
    item = item_type
    return eval(f"f'{template_input}'")

In [11]:
def chance_calculator(prob_occurence):
    fail_prob = 1 - prob_occurence
    tot_chance = fail_prob
    times = 1
    while tot_chance >= prob_occurence:
        tot_chance = fail_prob ** times
        times += 1
    return times

In [12]:
def get_currency_data():
    currencies = ["Currency"]
    currency_price_data = {}
    raw_currency_data = {}

    for currency in currencies:
        raw_currency_data[currency] = requests.get(template_replace(base_currency_url, currency)).json()
        
    for currency in raw_currency_data["Currency"]["lines"]:
        currency_price_data[currency["currencyTypeName"]] = {
           "chaosValue": currency["chaosEquivalent"]
        }
    
    return pd.DataFrame.from_dict(currency_price_data, orient="index")

In [17]:
def get_item_data():
    items = ["Scarab", "DeliriumOrb", "Essence"]
    item_price_data = {}
    raw_item_data = {}

    for item in items:
        raw_item_data[item] = requests.get(template_replace(base_item_url, item)).json()
    
    for category, data in raw_item_data.items():
        for item in data.get("lines"):
            item_price_data[item["name"]] = {
                "chaosValue": item["chaosValue"],
                "exaltedValue": item["exaltedValue"],
                "divineValue": item["divineValue"]
            }

    return pd.DataFrame.from_dict(item_price_data, orient="index")

In [18]:
currency_df = get_currency_data()

In [25]:
item_df = get_item_data()

In [19]:
harvest_df = currency_df.filter(like='Lifeforce', axis=0)
delirium_df = item_df.filter(like='Delirium Orb', axis=0)
scarab_df = item_df.filter(like='Scarab', axis=0)
essence_df = item_df.filter(like='Essence', axis=0)

In [63]:
#refactor this calculation to consider harvest price for better accuracy
delirium_df["chaosValue"]

Skittering Delirium Orb        33.0
Diviner's Delirium Orb         21.0
Fine Delirium Orb              11.0
Cartographer's Delirium Orb    10.0
Thaumaturge's Delirium Orb     10.0
Fragmented Delirium Orb         9.0
Singular Delirium Orb           9.0
Armoursmith's Delirium Orb      9.0
Foreboding Delirium Orb         9.0
Amorphous Delirium Orb          9.0
Blighted Delirium Orb           9.0
Timeless Delirium Orb           9.0
Whispering Delirium Orb         9.0
Imperial Delirium Orb           8.0
Jeweller's Delirium Orb         8.0
Abyssal Delirium Orb            8.0
Obscured Delirium Orb           8.0
Fossilised Delirium Orb         8.0
Blacksmith's Delirium Orb       8.0
Name: chaosValue, dtype: float64

In [56]:
max_value_delirium = delirium_df["chaosValue"].max()

In [62]:
primal_lifeforce_unit_price = harvest_df.at["Primal Crystallised Lifeforce", 'chaosValue']
vivid_lifeforce_unit_price = harvest_df.at["Vivid Crystallised Lifeforce", 'chaosValue']
wild_lifeforce_unit_price = harvest_df.at["Wild Crystallised Lifeforce", 'chaosValue']

In [113]:
def delirium_profit_calc():
    min_value_delirium = delirium_df["chaosValue"].min()
    
    max_stack_size = 10
    min_profit_margin = 20
    base_cost_delirium_orbs = min_value_delirium * 10
    crafting_try_cost = 30 * primal_lifeforce_unit_price * max_stack_size
    
    min_craft_cost = min_value_delirium * max_stack_size + crafting_try_cost
    profit_orbs = delirium_df[delirium_df["chaosValue"] * max_stack_size >= min_craft_cost + min_profit_margin]
    
    chance_to_hit = len(profit_orbs) / (len(delirium_df) - 1)
    worst_case_tries = chance_calculator(chance_to_hit)
    
    best_scenario = base_cost_delirium_orbs + crafting_try_cost * 1
    avg_scenario = base_cost_delirium_orbs + crafting_try_cost * (1 / chance_to_hit)
    worst_scenario = base_cost_delirium_orbs + crafting_try_cost * worst_case_tries
    
    costs = {
        "Best Scenario": best_scenario,
        "Avg Scenario": avg_scenario, 
        "Worst Scenario": worst_scenario
    }
    
    for i in range(len(profit_orbs), 0, -1):
        avg_profit = profit_orbs["chaosValue"].sum() * 10 / len(profit_orbs)
    
        print(f"Calculating profit when targetting {profit_orbs.head(i).index.tolist()}")

        for scenario, cost in costs.items():
            profit = avg_profit - cost
            print(f"""{scenario} cost: {cost:.0f} C. 
                Orbs: {base_cost_delirium_orbs} C.
                Crafting try: {crafting_try_cost} C.
                Profit: {profit:.0f} C!\n""")
        print("\n")

In [ ]:
delirium_profit_calc()